In [1]:
import tensorflow as tf
from tensorflow import keras
import gym
import random
'''print(tf.__version__)
assert tf.config.list_physical_devices('GPU')
print('hello')'''
env = gym.make('SpaceInvaders-v0')
h,w,c = env.observation_space.shape
actions = env.action_space.n

In [2]:
actions
print(env.unwrapped.get_action_meanings())

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [24]:
actions

6

In [46]:
episodes  = 5
for episode in range(1,episodes+1):
    done = False
    state = env.reset()
    score = 0
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state,reward,done,info = env.step(action)
        score+=reward
    print('episode {} score: {}'.format(episode,score))
env.close()

episode 1 score: 185.0
episode 2 score: 50.0
episode 3 score: 50.0
episode 4 score: 105.0
episode 5 score: 30.0


In [31]:
def build_model(h,w,c,actions):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32,(7,7),strides = (3,3),activation='relu',input_shape=(3,h,w,c) ))
    model.add(keras.layers.Conv2D(64,(5,5), activation = 'relu' ))
    model.add(keras.layers.Conv2D(64,(3,3), activation = 'relu' ))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256,activation = 'relu'))
    model.add(keras.layers.Dense(128,activation = 'relu'))
    model.add(keras.layers.Dense(actions,activation = 'linear'))
    return model

In [32]:
model = build_model(h,w,c,actions)
model.summary()

ValueError: Input 0 of layer conv2d_7 is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: [None, 3, 210, 160, 3]

In [5]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [14]:
def build_agent(model,actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),attr = 'eps',value_max = 1., value_min = .1, value_test = .2, nb_steps = 10000)
    memory = SequentialMemory(limit = 1000,window_length = 3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,enable_dueling_network = True, dueling_type = 'avg', nb_actions = actions, nb_steps_warmup = 1000)
    return dqn

In [15]:
dqn = build_agent(model,actions)
dqn.compile(keras.optimizers.Adam(lr = 1e-4))
dqn.fit(env, nb_steps = 10000,visualize = False)

Training for 10000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (1, 3, 210, 160, 3)